 # Create a Twisted Interface with Commensurate Lattices
 Use commensurate lattice matching algorithm to create twisted interfaces between two materials.
 
 <h2 style="color:green">Usage</h2>
 1. Make sure to select Input Materials (in the outer runtime) before running the notebook.
 1. Set notebook parameters in cell 1.1. below (or use the default values).
 1. Set material parameters for the substrate and film in cell 2.1. (or use default).
 1. Set interface parameters in cell 3.1. (or use default).
 1. Click "Run" > "Run All" to run all cells. 
 1. Wait for the run to complete (depending on the parameters can take a few min). 
 1. Scroll down to view results. 
 
 ## Summary
 1. Prepare the Environment: Set up the notebook and install packages, preview the input materials
 1. Create and visualize the initial materials
 1. Generate twisted interfaces with commensurate lattice matching
 1. Select the interface with the desired twist angle and visualize it
 
 ## Notes
 1. We perform commensurate lattice matching to find valid supercells that achieve the desired twist angle.
 1. When the matching is finished, interfaces with angles close to the target are presented.
 1. The algorithm searches for supercell matrices within specified size limits.
 1. For more information, see [Introduction](Introduction.ipynb)


 ## 1. Prepare the Environment
 ### 1.1. Set up the notebook 
 Set the following flags to control the notebook behavior 


In [28]:
# Material selection and basic parameters
FILM_INDEX = 1  # Index in the list of materials, to access as materials[FILM_INDEX]
SUBSTRATE_INDEX = 1  # Can be None to use same material as film

# Twisted interface parameters
TARGET_TWIST_ANGLE = 17.5  # in degrees
INTERFACE_DISTANCE = 3.0  # in Angstrom
INTERFACE_VACUUM = 20.0  # in Angstrom

# Search algorithm parameters
MAX_REPETITION =6  # Maximum supercell size to search
ANGLE_TOLERANCE = 0.5  # in degrees
RETURN_FIRST_MATCH = True  # If True, returns first solution within tolerance

# Visualization parameters
SHOW_INTERMEDIATE_STEPS = True
VISUALIZE_REPETITIONS = [3, 3, 1]


 ### 1.2. Import required packages


In [29]:
from mat3ra.made.material import Material
from mat3ra.made.tools.build.interface.builders import (
    CommensurateLatticeTwistedInterfaceBuilder,
    CommensurateLatticeTwistedInterfaceBuilderParameters,
    TwistedInterfaceConfiguration,
)
from mat3ra.made.tools.build.supercell import create_supercell
from utils.jupyterlite import get_data
from utils.visualize import visualize_materials
import plotly.graph_objs as go
import numpy as np


 ### 1.3. Load and preview input materials


In [30]:
# Get the input materials data
get_data("materials_in", globals())
materials = list(map(Material, globals()["materials_in"]))



0: Data from 0-Ni has been read successfully.
1: Data from 1-Graphene has been read successfully.
2: Data from 10-Al2O3, Sapphire, RHL (R-3c) 3D (Bulk), mp-1143 (1) has been read successfully.
3: Data from 11-ZnO, Zinc Oxide, HEX (P6_3mc) 3D (Bulk), mp-2133 has been read successfully.
4: Data from 12-Cd4 Te4 has been read successfully.
5: Data from 13-Si4 C4 has been read successfully.
6: Data from 14-GaN, Gallium Nitride, HEX (P6_3mc) 3D (Bulk), mp-804 has been read successfully.
7: Data from 15-WS2, Tungsten Disulfide, HEX (P-6m2) 2D (Monolayer), 2dm-3749 has been read successfully.
8: Data from 16 - In4 P4 has been read successfully.
9: Data from 4-Te2Mo has been read successfully.
10: Data from 5-HfO2 has been read successfully.
11: Data from 6-Ni4(110), termination Ni_Pmmm_2, Slab, Terrace, 1 steps, [2 0 0] has been read successfully.
12: Data from 7-Ag4 has been read successfully.
13: Data from 8-Si, Silicene, HEX (P-3m1) 2D (Monolayer), 2dm-5934 has been read successfully.
14: D

 ## 2. Prepare Materials
 ### 2.1. Select and visualize initial materials


In [31]:
# Select materials
film = materials[FILM_INDEX]
substrate = materials[SUBSTRATE_INDEX] if SUBSTRATE_INDEX is not None else film

print("\nSelected materials:")
print(f"Film: {film.formula}")
print(f"Substrate: {substrate.formula}")

if SHOW_INTERMEDIATE_STEPS:
    print("\nVisualizing film:")
    visualize_materials(film, repetitions=VISUALIZE_REPETITIONS)
    if substrate is not film:
        print("\nVisualizing substrate:")
        visualize_materials(substrate, repetitions=VISUALIZE_REPETITIONS)



Selected materials:
Film: None
Substrate: None

Visualizing film:


GridBox(children=(VBox(children=(Label(value='C2 - Material - rotation: 0x,0y,0z', layout=Layout(align_self='c…

 ## 3. Generate Twisted Interface
 ### 3.1. Set up interface configuration and builder


In [32]:
# Create configuration
config = TwistedInterfaceConfiguration(
    film=film,
    substrate=film,
    twist_angle=TARGET_TWIST_ANGLE,
    distance_z=INTERFACE_DISTANCE
)

# Set up builder parameters
params = CommensurateLatticeTwistedInterfaceBuilderParameters(
    max_repetition_int=MAX_REPETITION,
    angle_tolerance=ANGLE_TOLERANCE,
    return_first_match=RETURN_FIRST_MATCH
)

# Create builder
builder = CommensurateLatticeTwistedInterfaceBuilder(build_parameters=params)


### 3.2. Generate and analyze interfaces


In [ ]:
# Generate interfaces
interfaces = builder.get_materials(config)

print(f"\nFound {len(interfaces)} possible interface(s)")
for i, interface in enumerate(interfaces):
    actual_angle = interface.metadata.get("actual_twist_angle", "unknown")
    print(f"\nInterface {i+1}:")
    print(f"Actual twist angle: {actual_angle}°")
    print(f"Number of atoms: {len(interface.basis.elements.ids)}")

# Plot solutions
def plot_interface_solutions(interfaces):
    data = []
    for i, interface in enumerate(interfaces):
        angle = interface.metadata.get("actual_twist_angle", 0)
        size = len(interface.basis.elements.ids)
        
        hover_text = (
            f"Interface {i+1}<br>"
            f"Angle: {angle:.2f}°<br>"
            f"Atoms: {size}<br>"
        )
        
        trace = go.Scatter(
            x=[angle],
            y=[size],
            text=[hover_text],
            mode="markers",
            hoverinfo="text",
            name=f"Interface {i+1}"
        )
        data.append(trace)
    
    layout = go.Layout(
        title="Twisted Interface Solutions",
        xaxis=dict(title="Twist Angle (°)"),
        yaxis=dict(title="Number of Atoms"),
        hovermode="closest"
    )
    
    fig = go.Figure(data=data, layout=layout)
    fig.show()

if len(interfaces) > 0:
    plot_interface_solutions(interfaces)


## 4. Select and Visualize Interface
### 4.1. Select interface based on criteria


In [ ]:
if len(interfaces) == 0:
    print("No interfaces found! Try adjusting the search parameters.")
else:
    # Select interface closest to target angle
    angles = [interface.metadata.get("actual_twist_angle", 0) for interface in interfaces]
    angle_diffs = [abs(angle - TARGET_TWIST_ANGLE) for angle in angles]
    best_index = angle_diffs.index(min(angle_diffs))
    
    selected_interface = interfaces[best_index]
    actual_angle = selected_interface.metadata.get("actual_twist_angle", "unknown")
    
    print(f"Selected interface {best_index + 1}:")
    print(f"Target angle: {TARGET_TWIST_ANGLE}°")
    print(f"Actual angle: {actual_angle}°")
    print(f"Number of atoms: {len(selected_interface.atoms)}")
    print(f"Cell volume: {selected_interface.lattice.volume:.2f} Å³")
    
    # Visualize the selected interface
    visualize_materials(selected_interface, repetitions=[1, 1, 1])


### 4.2. Export selected interface


In [ ]:
if 'selected_interface' in locals():
    # Add vacuum if specified
    if INTERFACE_VACUUM > 0:
        selected_interface.lattice.vector_arrays[2][2] = INTERFACE_VACUUM
    
    # Export the interface
    selected_interface.to_file("twisted_interface.json")
    print("Interface exported to 'twisted_interface.json'")
    
    # Print final statistics
    print(f"\nFinal interface properties:")
    print(f"Twist angle: {actual_angle}°")
    print(f"Cell vectors:\n{selected_interface.lattice.vector_arrays}")
    print(f"Number of atoms: {len(selected_interface.atoms)}")